# Import

In [123]:
%load_ext autoreload
%autoreload 2

from pprint import pprint 
import subprocess
import plotly
import pandas as pd
from datetime import datetime 
import plotly.figure_factory as ff
import plotly.express as px
import os
import re
import json

from sflow import utils
from sflow.utils import get_bash_output
from sflow.slurm import SlurmClient, LogProcessor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Client APIs

In [3]:
sl = SlurmClient()
sl.get_info()

,STATE,NODES,NODELIST
0,drain*,1,h100-st-p548xlarge-288
1,drain,9,"h100-st-p548xlarge-[114,124,177,185,196,255,42..."
2,mix,129,"h100-st-p548xlarge-[0-49,59-73,110-112,115-119..."
3,alloc,132,"h100-st-p548xlarge-[50-51,156-165,197-210,213-..."
4,idle,179,"h100-st-p548xlarge-[52-58,74-109,113,120-123,1..."


In [4]:
sl.get_nodes_by_state("idle")

[52,
 53,
 54,
 55,
 56,
 57,
 58,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 113,
 120,
 121,
 122,
 123,
 125,
 126,
 137,
 138,
 152,
 153,
 154,
 155,
 166,
 167,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 211,
 212,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 256,
 257,
 258,
 275,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 379,
 380,
 424,
 425,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 442,
 

In [5]:
sl.get_queue().sort_values(by=['NODES'])

,JOBID,NAME,USER,ST,START_TIME,PARTITION,NODES,NODELIST
0,2634,bash,wangyanghan,R,2024-07-09T17:54:25,q1,1,h100-st-p548xlarge-168
19,3704,tranx,tranx,R,2024-07-15T15:09:15,q1,1,h100-st-p548xlarge-7
15,3100,bash,liuyanga,R,2024-07-11T18:26:20,q1,1,h100-st-p548xlarge-139
14,3133,1Banneal,liuyanga,R,2024-07-11T22:07:14,q1,1,h100-st-p548xlarge-21
13,3251,synthesis,wangyanghan,R,2024-07-12T17:06:29,q1,1,h100-st-p548xlarge-240
12,2608,1Bsft,liuyanga,R,2024-07-09T13:55:06,q1,1,h100-st-p548xlarge-5
11,3631,test2,liuyanga,R,2024-07-14T20:22:03,q1,1,h100-st-p548xlarge-3
9,3319,test,liuyanga,R,2024-07-12T19:18:54,q1,1,h100-st-p548xlarge-0
10,3655,distill,liuyanga,R,2024-07-15T04:19:33,q1,1,h100-st-p548xlarge-0
7,3462,bash,yfzhuang,R,2024-07-13T00:55:59,q1,1,h100-st-p548xlarge-2


# Job History

In [22]:
current_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
print(f"Current time: {current_time}")

df = sl.get_job_hist_table("2024-06-01")
df['End'] = df['End'].apply(lambda x: current_time if x == "Unknown" else x)


df1 = df[['User', 'JobID', 'JobName', 'State',
          'Start', 'End', 'NNodes', 'Elapsed_hr']]
df1['NNodes'] = 10*(df1['NNodes']//10).astype(int)

# map NNodes to the closest 10's, e.g., 2->0, 17 -> 10
# filter out all jobs using less than 10 nodes
df1 = df1[df1.NNodes >= 10]

# filter out all jobs that were cancelled
# df1 = df1[df1.State != 'CANCELLED+']
df1 = df1[df1.State.isin(['RUNNING', 'FAILED', 'COMPLETED'])]

# filter

df1.head()

Current time: 2024-07-15T15:28:34


,User,JobID,JobName,State,Start,End,NNodes,Elapsed_hr
45,jiazhi,46,vite_336,FAILED,2024-06-06T23:50:07,2024-06-06T23:55:21,10,0.08
46,jiazhi,47,vite_336,FAILED,2024-06-07T00:02:16,2024-06-07T00:06:35,10,0.07
50,jiazhi,51,vite_336,FAILED,2024-06-07T00:15:59,2024-06-07T00:17:02,10,0.02
53,jiazhi,54,vite_336,FAILED,2024-06-07T00:25:39,2024-06-07T00:27:28,190,0.02
55,jiazhi,56,vite_336,FAILED,2024-06-07T00:35:45,2024-06-07T00:38:00,190,0.03


In [20]:
df1.NNodes.unique()

array([ 10,  20, 220, 230, 240, 250, 260, 270, 280, 290, 300, 120,  60,
       360,  30, 160, 140,  40,  90, 180, 190, 330, 320, 100])

In [27]:
fig = px.timeline(
    df1, x_start="Start", x_end="End", y="NNodes", color="State",
    color_discrete_map={'RUNNING': 'blue',
                        'FAILED': 'red', 'COMPLETED': 'green'},
    hover_data=['Elapsed_hr', 'JobID', 'JobName', 'User'],
)
fig.show()

In [28]:
html_string = fig.to_html(
    include_plotlyjs=True,  # include the Plotly.js library in the HTML
    default_width='100%',  # set the default width of the plot to 100%
    default_height=500  # set the default height of the plot to 500 pixels
)
# save the HTML string to a file
with open('/fsx_0/user/tranx/cluster/stats/job_status.html', 'w') as f:
    f.write(html_string)

# Job history - anonymized

In [103]:
current_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
print(f"Current time: {current_time}")

df = sl.get_job_hist_table("2024-06-01")
df['End'] = df['End'].apply(lambda x: current_time if x == "Unknown" else x)


df2 = df[['User', 'JobID', 'JobName', 'State',
          'Start', 'End', 'NNodes', 'Elapsed_hr']]
df2['NNodes'] = 10*(df2['NNodes']//10).astype(int)

# map NNodes to the closest 10's, e.g., 2->0, 17 -> 10
# filter out all jobs using less than 10 nodes
df2 = df2[df2.NNodes >= 10]

# filter out all jobs that were cancelled
# df1 = df1[df1.State != 'CANCELLED+']
df2 = df2[df2.State.isin(['RUNNING', 'FAILED', 'COMPLETED'])]

# exclude nccl test
df2 = df2[~df2['JobName'].str.startswith("nccl")]

# anonymize
job_names_map = {name: f"JobName-{i}" for i,
                 name in enumerate(df2.JobName.unique())}
user_map = {name: f"User-{i}" for i, name in enumerate(df2.User.unique())}

df2['JobName'] = df2['JobName'].map(job_names_map)
df2['User'] = df2['User'].map(user_map)

df2.head()

Current time: 2024-07-17T12:10:37


,User,JobID,JobName,State,Start,End,NNodes,Elapsed_hr
45,User-0,46,JobName-0,FAILED,2024-06-06T23:50:07,2024-06-06T23:55:21,10,0.08
46,User-0,47,JobName-0,FAILED,2024-06-07T00:02:16,2024-06-07T00:06:35,10,0.07
50,User-0,51,JobName-0,FAILED,2024-06-07T00:15:59,2024-06-07T00:17:02,10,0.02
53,User-0,54,JobName-0,FAILED,2024-06-07T00:25:39,2024-06-07T00:27:28,190,0.02
55,User-0,56,JobName-0,FAILED,2024-06-07T00:35:45,2024-06-07T00:38:00,190,0.03


In [102]:
fig_anon = px.timeline(
    df2, x_start="Start", x_end="End", y="NNodes", color="State",
    color_discrete_map={'RUNNING': 'blue',
                        'FAILED': 'red', 'COMPLETED': 'green'},
    hover_data=['Elapsed_hr', 'JobID', 'JobName', 'User'],
)
fig_anon.show()

In [47]:
html_string = fig_anon.to_html(
    include_plotlyjs=True,  # include the Plotly.js library in the HTML
    default_width='100%',  # set the default width of the plot to 100%
    default_height=700  # set the default height of the plot to 500 pixels
)
# save the HTML string to a file
with open('/fsx_0/user/tranx/cluster/stats/job_status_july15_anon.html', 'w') as f:
    f.write(html_string)

In [95]:
# p95 - elapsed time
df2 = df2[df2.NNodes < 200]
p95_day = df2.groupby('NNodes')['Elapsed_hr'].quantile(0.95)
p95_day = round(p95_day/24, 2)

fig_p95 = px.bar(p95_day)
fig_p95.update_layout(title='P95 Job Duration versus Job Scale',
                      xaxis_title='Number of nodes',
                      yaxis_title='P95 Job Duration (days)')
# Show the chart
fig_p95.show()

In [96]:
html_string = fig_p95.to_html(
    include_plotlyjs=True,  # include the Plotly.js library in the HTML
    default_width='100%',  # set the default width of the plot to 100%
    default_height=500  # set the default height of the plot to 500 pixels
)
# save the HTML string to a file
with open('/fsx_0/user/tranx/cluster/stats/p95_job_duration_july15.html', 'w') as f:
    f.write(html_string)

In [71]:
df2_ratio

NNodes
10     85.0
20      5.0
30     12.0
40      5.0
50      1.0
60      1.0
70      1.0
80      3.0
90      3.0
100     3.0
110     1.0
120     4.0
140     2.0
160     2.0
180     5.0
190     2.0
dtype: float64

In [91]:
failure_rate = {}
for i in sorted(df2.NNodes.unique()):
    r = df2[(df2.NNodes == i) & (df2.State == 'FAILED')
            ].shape[0]/df2[(df2.NNodes == i)].shape[0]
    failure_rate[i] = round(r, 2)

failure_rate

{10: 0.5,
 20: 0.64,
 30: 0.74,
 40: 0.64,
 50: 1.0,
 60: 1.0,
 70: 1.0,
 80: 0.82,
 90: 0.87,
 100: 0.83,
 110: 1.0,
 120: 0.97,
 140: 0.98,
 160: 0.98,
 180: 0.64,
 190: 0.9}

In [93]:
px.line(x=failure_rate.keys(), y=failure_rate.values())

# Log parsing

In [120]:
sl = SlurmClient()

job_hist = sl.get_job_hist_json(start_date="2024-07-01")
# hist = hist[:100]

In [121]:
hist_info = []
for h in job_hist:
    info = LogProcessor.parse_job_info(h)
    if info is not None:
        hist_info.append(info)

-------
{'elapsed_seconds': 1555,
 'end_time': '2024-07-01 09:51:38',
 'job_id': 1573,
 'log_info': {'found_error_patterns': ['RendezvousTimeoutError',
                                       'Watchdog caught collective operation '
                                       'timeout'],
              'train_epoch': 17},
 'node_count': 128,
 'sbatch_config': {'cpus-per-task': 96,
                   'error': '/fsx_0/user/jiazhi/logs/error_1573.txt',
                   'exclude': 'h100-st-p548xlarge-71,h100-st-p548xlarge-7,h100-st-p548xlarge-70,h100-st-p548xlarge-69,h100-st-p548xlarge-68,h100-st-p548xlarge-73,h100-st-p548xlarge-72,h100-st-p548xlarge-241,h100-st-p548xlarge-242,h100-st-p548xlarge-243,h100-st-p548xlarge-211,h100-st-p548xlarge-330,h100-st-p548xlarge-331,h100-st-p548xlarge-165',
                   'gpus-per-task': 8,
                   'job-name': 'vitg_224',
                   'mem': 0,
                   'nodes': 128,
                   'ntasks': 128,
                   'output': 

In [122]:
hist_info

[{'job_id': 1573,
  'node_count': 128,
  'user': 'jiazhi',
  'state': 'FAILED',
  'log_info': {'train_epoch': 17,
   'found_error_patterns': ['RendezvousTimeoutError',
    'Watchdog caught collective operation timeout']},
  'start_time': '2024-07-01 09:25:43',
  'end_time': '2024-07-01 09:51:38',
  'elapsed_seconds': 1555,
  'sbatch_config': {'job-name': 'vitg_224',
   'nodes': 128,
   'ntasks': 128,
   'gpus-per-task': 8,
   'cpus-per-task': 96,
   'mem': 0,
   'output': '/fsx_0/user/jiazhi/logs/output_1573.txt',
   'error': '/fsx_0/user/jiazhi/logs/error_1573.txt',
   'time': '168:00:00',
   'qos': 'ar-ai-hipri',
   'exclude': 'h100-st-p548xlarge-71,h100-st-p548xlarge-7,h100-st-p548xlarge-70,h100-st-p548xlarge-69,h100-st-p548xlarge-68,h100-st-p548xlarge-73,h100-st-p548xlarge-72,h100-st-p548xlarge-241,h100-st-p548xlarge-242,h100-st-p548xlarge-243,h100-st-p548xlarge-211,h100-st-p548xlarge-330,h100-st-p548xlarge-331,h100-st-p548xlarge-165'}},
 {'job_id': 1675,
  'node_count': 128,
  'us

In [125]:
file = '/fsx_0/user/tranx/cluster/stats/job_history_since_20240701.json'
with open(file, "w") as f:
    json.dump(hist_info, f)

In [126]:
with open(file, "r") as f:
    data = json.load(f)